MLA 2024 Assignment 2 (SVMs)

Name1 MTNR1
Name2 MTNR2
Name3 MTNR3

# Definition of input files

In [ ]:
#These filename need to be defined:

#  current dataset directory : data/alldigits.csv

digits_train_filename = 'data/traindigits.csv'

# Your sections ...

# Calc1: Read data

In [ ]:
import numpy as np


digits_train = np.loadtxt(digits_train_filename, delimiter=',', skiprows=1)

NameError: name 'digits_train_filename' is not defined

# Explore

In [3]:
import matplotlib.pyplot as plt

px = digits_train[:,400]
px.sort()
plt.plot(px)

NameError: name 'digits_train' is not defined

# Calc2: Naive Bayes

In [4]:
from sklearn.naive_bayes import GaussianNB

X = digits_train[:, :-1]
y = digits_train[:, -1]

gnb = GaussianNB().fit(X,y)

KeyboardInterrupt: 

# ...

# Final classifiers

In [11]:
#This classifier needs to be defined:
digits_SVM = gnb